In [1]:
from model import *
import torch
def mem(device=None):
    print("Max Memory occupied by tensors: ", torch.cuda.max_memory_allocated(device=device))
    print("Max Memory Cached: ", torch.cuda.max_memory_reserved(device=device))
    print("Current Memory occupied by tensors: ", torch.cuda.max_memory_allocated(device=device))
    print("Current Memory cached occupied by tensors: ", torch.cuda.memory_reserved(device=device))

In [2]:
mem()

Max Memory occupied by tensors:  0
Max Memory Cached:  0
Current Memory occupied by tensors:  0
Current Memory cached occupied by tensors:  0


In [3]:
class Segmentator(nn.Module):
    def __init__(self, in_channels=3, layers_settings: List[Tuple] = None, activation_function: Optional[Callable[..., nn.Module]] = None) -> None:
        super(Segmentator, self).__init__()
        if not layers_settings:
            layers_settings = [ # t c n s
                    (1, 16, 1, 1),
                    (6, 24, 2, 2),
                    (6, 32, 3, 1),
                    (6, 64, 4, 1),
                    (6, 120, 1, 1),
                    (6, 320, 1, 1),
                    (6, 1, 1, 1)
                ]
        layers = []
        for t, c, n, s in layers_settings:
            for i in range(n):
                layer = InvertedResidual(in_channels, c, s, t, activation_function=activation_function)
                in_channels = c
                layers.append(layer)
        self.layers = nn.Sequential(*layers)
        self.sig = nn.Sigmoid()
    
    def forward(self, input: Tensor):
        small_mask = self.sig(self.layers(input))
        B, C, H, W = input.shape
        big_mask = F.interpolate(small_mask, size=(H, W))
        return big_mask

In [4]:
n = torch.zeros(1).cuda()
mem()

Max Memory occupied by tensors:  512
Max Memory Cached:  2097152
Current Memory occupied by tensors:  512
Current Memory cached occupied by tensors:  2097152


In [10]:
32/8

4.0